**Анализ данных**

В файлах events и levels по игре Bewitching Forest. Желательно установить и поиграть, доступна на Google Play. В таблицах данные по событиям за первые 24 часа жизни пользователя.

levels: 
* uid - айди юзера
* created_at - дата/время события
* event_name - события, связанные с уровнем (начало, конец, выигрыш, проигрыш и тд)
* level_move - кол-во ходов уровня
* Number_of_used_moves - Количество использованных ходов
* target - Количество целей в начале уровня
* remaining_targets - Счетчик оставшихся целей
* reg_dt	- дата регистрации
* churn - отвал после первых 24 часов 1(да), 0(нет)

events:
Тут события, которые могут быть внутри уровня. 
* Расход жизней на уровне -	spent_replay
* Расход бесконечных жизней на уровне - spent_unlimited_lives_replay
* Покупка жизней - spent_life_purchase
* Покупка ходов в гринде - spent_turns_purchase
* Покупка пака хинтов - pent_hint_pack_purchase
* Покупка пака бустеров - spent_booster_pack_purchase
* Расход хинта на уровне - spent_level

Задача: проанализировать отвалы по уровням и событиям внутри уровней. 

In [ ]:
!pip install plotly==5.2.1

     |████████████████████████████████| 21.8 MB 1.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


# **0. Предобработка и загрузка данных.**

Обновим версию plotly до последней, чтоб использовать все возможные особенности. Также импортируем основные библиотеки.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

# чтобы не получать лишние предупреждения
pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
df_events = pd.read_csv('/gdrive/MyDrive/Mail test/events.csv')
df_levels = pd.read_csv('/gdrive/MyDrive/Mail test/levels.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Посмотрим на файл levels.

In [ ]:
df_levels.head(1)

,uid,created_at,event_name,level_move,Number_of_used_moves,target,remaining_targets,reg_dt,churn
0,5bc9921d-5a3c-41c7-9a43-65496e6ecebe,2021-03-15 18:56:12,battle_start,20,NaN,NaN,NaN,2021-03-13 00:05:17,1


In [ ]:
df_levels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319297 entries, 0 to 319296
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   uid                   319297 non-null  object 
 1   created_at            319297 non-null  object 
 2   event_name            319297 non-null  object 
 3   level_move            319297 non-null  object 
 4   Number_of_used_moves  156433 non-null  float64
 5   target                156433 non-null  object 
 6   remaining_targets     153768 non-null  float64
 7   reg_dt                319297 non-null  object 
 8   churn                 319297 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 21.9+ MB


Необходимо перевести временные колонки в формат datetime.

In [ ]:
df_levels['created_at'] = pd.to_datetime(df_levels['created_at'])
df_levels['reg_dt'] = pd.to_datetime(df_levels['reg_dt'])

Точно также выведем основную информацию по файлу events.

In [ ]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50494 entries, 0 to 50493
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         50494 non-null  object
 1   event_name  50494 non-null  object
 2   created_at  50494 non-null  object
 3   reg_dt      50494 non-null  object
 4   churn       50494 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [ ]:
df_events.head(2)

,uid,event_name,created_at,reg_dt,churn
0,ad1be1b6-e696-42d6-ac03-99be5803a14b,spent_replay,2021-03-13 03:38:24,2021-03-13 03:36:08,1
1,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,spent_replay,2021-03-13 08:43:37,2021-03-13 08:41:28,0


# **1. Добавление стартовых разделов.**

Добавим столбец указывающий время прохождения уровня, как победной сессии, так и обычной. Для этого отфильтруем данные по времени событий и добавим прошедшее время между двумя строками с battle_start для каждого юзера. Второй battle_start не включен в рассмотрение.

In [ ]:
df_levels = df_levels.sort_values(by='created_at', ascending=True).reset_index()

In [ ]:
index_start = [row
               for row in range(len(df_levels))
               if df_levels.loc[row, 'event_name'] == 'battle_start']


In [ ]:
user_list = list(df_levels.uid.unique())

In [ ]:
for user in user_list:
  df_temp = df_levels[df_levels['uid'] == user]
  index_start = [row
                  for row in list(df_temp.index)
                  if df_temp.loc[row, 'event_name'] == 'battle_start']
  index_temp = list(df_temp.index)
  for i in range(len(index_start)):
    if i == len(index_start)-1:
      df_levels.loc[index_start[i], 'delta_time'] = df_temp.loc[index_temp[-1],'created_at'] - \
                             df_temp.loc[index_start[i],'created_at']
    else:
      index_temp_1 = index_temp.index(index_start[i + 1])
      df_levels.loc[index_start[i], 'delta_time'] = df_temp.loc[index_temp[index_temp_1 - 1],'created_at'] - \
                             df_temp.loc[index_start[i],'created_at']    

Помимо этого добавим для каждой строки battle_win значение уровня, который был пройден.

In [ ]:
df_levels_new = pd.DataFrame(columns=['index', 'uid', 'created_at', 'event_name',
 'level_move', 'Number_of_used_moves', 'target', 'remaining_targets',
 'reg_dt', 'churn', 'level'])

In [ ]:
for user in user_list:
  count = 1
  df_temp = df_levels[df_levels['uid'] == user].sort_values(by='created_at')
  df_temp['level'] = np.nan
  for row in list(df_temp.index):
    if df_temp.loc[row, 'event_name'] == 'battle_win':
      df_temp.loc[row, 'level'] = count
      count += 1
  df_temp['level'].fillna(method='bfill', inplace=True)
  df_levels_new = pd.concat([df_levels_new,df_temp])

In [ ]:
df_levels_new = df_levels_new.sort_values(by='created_at')
df_levels_new['level'].fillna(0, inplace=True) 
df_levels_new

,index,uid,created_at,event_name,level_move,Number_of_used_moves,target,remaining_targets,reg_dt,churn,level,delta_time
0,24,ad1be1b6-e696-42d6-ac03-99be5803a14b,2021-03-13 03:38:18,battle_start,20,NaN,NaN,NaN,2021-03-13 03:36:08,0,1.0,0 days 00:01:52
1,25,ad1be1b6-e696-42d6-ac03-99be5803a14b,2021-03-13 03:40:10,battle_win,20,8.0,73,112.0,2021-03-13 03:36:08,0,1.0,NaT
2,26,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:43:15,battle_start,20,NaN,NaN,NaN,2021-03-13 08:41:28,1,1.0,0 days 00:02:22
3,27,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:45:37,battle_win,20,6.0,73,142.0,2021-03-13 08:41:28,1,1.0,NaT
4,28,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:46:39,battle_start,20,NaN,NaN,NaN,2021-03-13 08:41:28,1,2.0,0 days 00:00:54
...,...,...,...,...,...,...,...,...,...,...,...,...
319292,228793,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:09:50,battle_start,29,NaN,NaN,NaN,2021-03-16 14:04:39,1,0.0,0 days 00:02:25
319293,228794,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:12:15,battle_loss,29,29.0,114,35.0,2021-03-16 14:04:39,1,0.0,NaT
319294,228795,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:12:19,battle_start,29,NaN,NaN,NaN,2021-03-16 14:04:39,1,0.0,0 days 00:00:00
319295,262394,1f822608-4036-47f4-b36d-7ac648c89103,2021-04-07 14:23:07,battle_start,23,NaN,NaN,NaN,2021-03-16 21:01:04,1,0.0,0 days 00:02:14


# **1.1 Выделение событий произошедших за первые 24 часа после регистрации.**

Посчитаем разницу между датой регистрации и каждым событием для пользователей, и добавим метрику указывающую было ли это до 24 часов или после.

In [ ]:

df_levels['timedelta'] = df_levels['created_at'] - df_levels['reg_dt']
df_levels['24_hour'] = [1
                        if df_levels.loc[row,'timedelta'].days < 1
                        else 2
                        for row in range(len(df_levels))]

Таким образом обрежем значения по столбцу df_levels['24_hour'] = 1.
Полученный дата сет df_24.

In [ ]:
df_24 = df_levels[df_levels['24_hour'] == 1]

Посмотрим на оставшийся кусок данных с первого дня по 3 день.

In [ ]:
df_3 = df_levels[df_levels['24_hour'] == 2]
df_3

,index,uid,created_at,event_name,level_move,Number_of_used_moves,target,remaining_targets,reg_dt,churn,delta_time,timedelta,24_hour
22336,1475,f6b63ace-9dc0-4244-ac97-2ee5df268408,2021-03-14 13:58:52,battle_start,25,NaN,NaN,NaN,2021-03-13 12:19:22,1,0 days 00:01:30,1 days 01:39:30,2
22381,1476,f6b63ace-9dc0-4244-ac97-2ee5df268408,2021-03-14 14:00:22,battle_loss,25,25.0,87,1.0,2021-03-13 12:19:22,1,NaT,1 days 01:41:00,2
22384,1477,f6b63ace-9dc0-4244-ac97-2ee5df268408,2021-03-14 14:00:26,battle_start,25,NaN,NaN,NaN,2021-03-13 12:19:22,1,0 days 00:01:17,1 days 01:41:04,2
22420,1478,f6b63ace-9dc0-4244-ac97-2ee5df268408,2021-03-14 14:01:43,battle_loss,25,25.0,87,1.0,2021-03-13 12:19:22,1,NaT,1 days 01:42:21,2
22423,1479,f6b63ace-9dc0-4244-ac97-2ee5df268408,2021-03-14 14:01:47,battle_start,25,NaN,NaN,NaN,2021-03-13 12:19:22,1,0 days 00:01:09,1 days 01:42:25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319292,228793,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:09:50,battle_start,29,NaN,NaN,NaN,2021-03-16 14:04:39,1,0 days 00:02:25,22 days 00:05:11,2
319293,228794,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:12:15,battle_loss,29,29.0,114,35.0,2021-03-16 14:04:39,1,NaT,22 days 00:07:36,2
319294,228795,04cb05e7-67e6-4a6e-a40f-657432f157ba,2021-04-07 14:12:19,battle_start,29,NaN,NaN,NaN,2021-03-16 14:04:39,1,0 days 00:00:00,22 days 00:07:40,2
319295,262394,1f822608-4036-47f4-b36d-7ac648c89103,2021-04-07 14:23:07,battle_start,23,NaN,NaN,NaN,2021-03-16 21:01:04,1,0 days 00:02:14,21 days 17:22:03,2


* Всего количество пользователей 2812, из них первые 24 часа было активно 2789, после 24 активно 1413.
* Общее число пользователей закончивших хотя бы один уровень 2723, из них первые 24 часа - 2689, после 24 - 1335.

In [ ]:
dataset_all=pd.DataFrame(columns=['title','count'])
dataset_all.loc['all users','count'] = len(df_levels.uid.unique())
dataset_all.loc['24 hour','count'] = len(df_24.uid.unique())
dataset_all.loc['more than 24 h','count'] = len(df_3.uid.unique())
dataset_all

,title,count
all users,NaN,2812
24 hour,NaN,2789
more than 24 h,NaN,1413


In [ ]:
dataset_players=pd.DataFrame()
dataset_players.loc['all users','count'] = len(df_levels.uid.unique())
dataset_players.loc['1 level','count'] = len(df_levels[df_levels['event_name'] == 'battle_win'].uid.unique())
dataset_players.loc['24 hour','count'] = len(df_24[df_24['event_name'] == 'battle_win'].uid.unique())
dataset_players.loc['more than 24 h','count'] = len(df_3[df_3['event_name'] == 'battle_win'].uid.unique())

In [ ]:
fig = go.Figure(go.Funnel(
    y = dataset_all.index,
    x = dataset_all['count'], textposition = "inside",
    textinfo = "value+percent previous"))

fig.update_layout(title = 'data for all users')
fig.show()

In [ ]:
fig = go.Figure(go.Funnel(
    y = dataset_players.index,
    x = dataset_players['count'], textposition = "inside",
    textinfo = "value+percent previous"))

fig.update_layout(title = 'data for playing users')
fig.show()

# **2. Выделение номера уровня, или количества пройденных игр.**

Посмотрим на уникальные event_name, чтоб выделить уровни, или количество пройденных игр пользователями.

In [ ]:
df_levels.event_name.unique()

array(['battle_start', 'battle_win', 'battle_loss', 'battle_backtrack'],
      dtype=object)

В ходе игры, если пользователь проходит уровень, то сразу переходит к следующему. Значит по значению event_name = battle_win можно определить на каком уровне пользователь находится.

Для этого построим сводную таблицу, где посчитаем сколько раз каждый из пользователей заканчивал уровень, крашил уровень, проигрывал и т.д. Кроме того узнаем до какого уровня пользователь дошел за 24 часа.
Точно информацию по игре не нашла, но вроде есть уровни выше 389. Так что дополнительно посмотрим на максимум полученного значения уровня.

Выделив все варианты названия события для пользователя построим в начале таблицу суммы всех событий и соединим потом с таблицей данных по максимальным уровням.

In [ ]:
df_levels.event_name.unique()

array(['battle_start', 'battle_win', 'battle_loss', 'battle_backtrack'],
      dtype=object)

In [ ]:
table_all_1 = df_levels.pivot_table(index='uid', columns='event_name', 
                                    values='churn',
                                    aggfunc='count', fill_value=0
                                    ).reset_index()
table_all_1['battle_crash'] = table_all_1['battle_start'] - \
                              table_all_1['battle_win'] - \
                              table_all_1['battle_loss'] - \
                              table_all_1['battle_backtrack']
table_all_1

event_name,uid,battle_backtrack,battle_loss,battle_start,battle_win,battle_crash
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,0,5,5,0
1,0018f5d2-d883-4ad2-9b96-29028069ee81,0,31,149,126,-8
2,00431e7e-d833-47c9-aae9-7ce21652902f,0,0,4,4,0
3,0060a2dd-18d8-4122-96ca-92ae699a1a52,1,103,207,93,10
4,0063c814-edef-4d88-b866-f39ac440c538,0,1,11,10,0
...,...,...,...,...,...,...
2807,ff958cbb-bd1a-4f53-a545-79ea2fba993e,0,0,14,12,2
2808,ff96e64e-8ebf-48e3-8d52-78f247468068,0,51,94,31,12
2809,ffc552f6-b662-4b62-bf3b-188cf5c23760,0,0,4,4,0
2810,ffdb1aaf-6c57-4fb7-a3a9-da40857f9cf8,0,5,25,17,3


In [ ]:
table_all_2 = df_levels[df_levels['event_name'
                    ] == 'battle_win'
                    ].pivot_table(index='uid', 
                                 aggfunc={'event_name': 'count',
                                          '24_hour':'max',
                                          'churn':'mean'}
                                ).reset_index().sort_values(by='event_name', 
                                                            ascending=False)
table_all_2 = table_all_2.rename(columns={'event_name': 'count_event'})
table_all = pd.merge(table_all_1, table_all_2, how='inner', on='uid')
table_all.head(2)

,uid,battle_backtrack,battle_loss,battle_start,battle_win,battle_crash,24_hour,churn,count_event
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,0,5,5,0,1,0,5
1,0018f5d2-d883-4ad2-9b96-29028069ee81,0,31,149,126,-8,2,1,126


Разделим таблицу на людей играющих только 24 часа и с 1 дня по 3.

In [ ]:
table_24_1 = df_levels[(df_levels['24_hour'
                    ] == 1)].pivot_table(index='uid', columns='event_name', 
                                    values='churn',
                                    aggfunc='count', fill_value=0
                                    ).reset_index()
table_24_1['battle_crash'] = table_24_1['battle_start'] - \
                              table_24_1['battle_win'] - \
                              table_24_1['battle_loss'] - \
                              table_24_1['battle_backtrack']
table_24_2 = df_levels[(df_levels['event_name'
                    ] == 'battle_win') &
                     (df_levels['24_hour'
                    ] == 1)
                    ].pivot_table(index='uid', 
                                 aggfunc={'event_name': 'count',
                                          'churn':'mean'}
                                ).reset_index().sort_values(by='event_name', 
                                                            ascending=False)
table_24_2 = table_24_2.rename(columns={'event_name': 'max_level'})
table_24 = pd.merge(table_24_1, table_24_2, how='inner', on='uid')
table_24

,uid,battle_backtrack,battle_loss,battle_start,battle_win,battle_crash,churn,max_level
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,0,5,5,0,0,5
1,0018f5d2-d883-4ad2-9b96-29028069ee81,0,11,54,43,0,1,43
2,00431e7e-d833-47c9-aae9-7ce21652902f,0,0,4,4,0,0,4
3,0060a2dd-18d8-4122-96ca-92ae699a1a52,0,38,98,57,3,1,57
4,0063c814-edef-4d88-b866-f39ac440c538,0,0,4,4,0,1,4
...,...,...,...,...,...,...,...,...
2684,ff958cbb-bd1a-4f53-a545-79ea2fba993e,0,0,14,12,2,0,12
2685,ff96e64e-8ebf-48e3-8d52-78f247468068,0,12,38,24,2,1,24
2686,ffc552f6-b662-4b62-bf3b-188cf5c23760,0,0,4,4,0,0,4
2687,ffdb1aaf-6c57-4fb7-a3a9-da40857f9cf8,0,0,5,4,1,1,4


In [ ]:
table_3 = table_all[table_all['24_hour'] == 2]
table_3['max_level'] = table_3['battle_win']
table_3.head(1)

,uid,battle_backtrack,battle_loss,battle_start,battle_win,battle_crash,24_hour,churn,count_event,max_level
1,0018f5d2-d883-4ad2-9b96-29028069ee81,0,31,149,126,-8,2,1,126,126


В данных возник выброс в виде того, что есть пользователи, у которых количество пройденных, проигранных и запущенных игр не равно друг другу. В данных таблицах это записывалась как краш игры. Но есть пользователи, где он отрицательный. Посмотрим на распределение получившегося столбца.

In [ ]:
px.box(table_all, y='battle_crash', x='churn')

**Интересный факт**
Меньше 25% всех пользователей столкнулись с тем, что количество так называемых крашей игр, или случаев когда количество запусков игры не равняется сумме винов, лузов или того, что игрок сдался.

# **3. Связь отвала пользователей с номером максимального уровня, до которого они смогли добраться.**

Построим полученный отвал пользователей для обеих групп, с указанием на 'churn' в виде цвета гистограммы.

In [ ]:
px.histogram(table_24[table_24['max_level'] <=100], x="max_level", color='churn',
            title='Рис.1 Гистограмма для пользователей, играющих только первые 24 часа', nbins=50)


In [ ]:
px.histogram(table_3[table_3['max_level'] <= 100], x="max_level", color='churn',
            title='Рис.2 Гистограмма для пользователей, играющих 1-3 дня', nbins=50)

In [ ]:
px.histogram(table_3[table_3['max_level'] > 100], x="max_level", color='churn',
            title='Рис.3 Гистограмма для пользователей, играющих 1-3 дня max_level>100')

**Выводы:**
* Выделяются уровни на которых отсеилось большинство игроков. Это 14-15 уровень и 28-29. 48-49 тоже выделяется, но его статистическая значимость меньше, так как выборка становится значительно уменьшается. Видно из рис. 1.
* Если согласно описанию таблицы, churn=1 это отвал игрока через 24 часа, то выделяется тенденция, что синяя гистограмма имеет распределение похожее на ступеньку. Это говорит о том, что отвал игроков происходит практически равномерно с 0 уровня по 48, не считая пиков. Видно из рис. 1.
* В то же время пользователи играющие после 24 часов прекращают проходить уровни начальной группы ( с 1 по 19 уровень), что видно из красной гистограммы. Видно из рис. 1.
* Странный факт, что все игроки играющие больше 1 дня имеют churn = 1 Это говорит о том, что все игроки играющие дольше забросили игру после 24 часов. Возможно не правильно дано описание этой колонки.
* Пользователи оставшиеся в игре больше 24 часов, чаще всего забрасывают игру после уровней с 10 по 49. Дальше уже виден плавный спад.

# **4. Разделение всех пользователей на когорты согласно пройденным уровням.**

Разделим пользователей на когорты по уровням, на которых они закончили играть по имеющимся данным. Отдельно рассмотрим данные за первые 24 часа в игре и с 1 по 3 день. Для удобства создадим функцию для расчета когорт.

В качестве разделения уровней выберем через каждые 15 уровней. Это связано с первым пиком на рис.1. Так как первая сложность и высокая вероятность того, что игроки забросят игру был на 15, потов районе 28-29. Таким образом проверим количество пользователей через каждые 15 уровней мы также зацепим эти пики.

Для удобства работы с указателем на группы, мы заполним столбец не интервала групп, а минимальным из когорт, т.е. 0, 15, 30 и т.д. 

In [ ]:
def kogort_15(table):
  for i in range(0, 415, 15):
    temp_data = table[(table['max_level'] >= i - 15) & (table['max_level'] <= i)]
    temp_index = list(temp_data.index)
    table.loc[temp_index, 'level_group'] = str(i - 15)# + ' - ' + str(i)
    table = table.reset_index(drop=True)
  return table

Для данных за первые 24 часа имеем:

In [ ]:
table_short_24 = kogort_15(table_24).pivot_table(index='level_group', 
                                                 values='churn', 
                                                 aggfunc='count')#lambda series: series.sum() / len(series) )
table_short_24 = table_short_24.reset_index()
table_short_24['level_group'] = table_short_24['level_group'].astype('int64')
table_short_24 = table_short_24.rename(columns={'churn': 'count_users'})
table_short_24

,level_group,count_users
0,0,1358
1,15,810
2,30,338
3,45,154
4,60,22
5,75,6
6,90,1


In [ ]:
table_short_3 = kogort_15(table_3).pivot_table(index='level_group', 
                                                 values='churn', 
                                                 aggfunc='count')
table_short_3 = table_short_3.reset_index()
table_short_3['level_group'] = table_short_3['level_group'].astype('int64')
table_short_3 = table_short_3.sort_values(by='level_group')
table_short_3 = table_short_3.rename(columns={'churn': 'count_users'})
table_short_3

,level_group,count_users
0,0,142
4,15,255
12,30,260
14,45,252
15,60,116
16,75,82
17,90,54
1,105,34
2,120,27
3,135,22


In [ ]:
fig = go.Figure(data = [go.Bar(name='24 hour', x=table_short_24['level_group'], 
                               y=table_short_24['count_users']), 
                        go.Bar(name='1-3 days', x=table_short_3['level_group'], 
                               y=table_short_3['count_users']), 
                        ])
fig.update_layout(barmode='group')
fig.show()

**Выводы:**

Из распределения пройденного максимального уровня видно, что за первые 24 часа чаще всего забрасывают игру люди остановившиеся на 0-15 и 15-30 уровнях.

Однако если игроки заходят в игру от 1 до 3 дней, то распределение максимального уровня сдвигается и это уже 15-65 уровни. 

ДОбавим процент забросивших игру для каждой из когорт. А именно подсчет churn. Здесь мы выведем только график для данных об 24 часах, так как мы видели ранее, что пользователи проводившие в игре свыше 1 дня всегда имеют churn=1.

In [ ]:
table_short_24_churn = kogort_15(table_24).pivot_table(index=['level_group'], 
                                                 values='churn', aggfunc=lambda series: series.sum() / len(series) )
table_short_24_churn = table_short_24_churn.reset_index()
table_short_24_churn

,level_group,churn
0,0,0.329161
1,15,0.659259
2,30,0.843195
3,45,0.902597
4,60,1.000000
5,75,1.000000
6,90,1.000000


In [ ]:
px.bar(table_short_24_churn, x='level_group', y='churn')

**Вывод:**

C 1 уровня до 59 происходит рост отвала пользователей и с 60 churn=1.

# **5. Связь прохождения уровней с типом бустера.**

Создадим таблицу числа всех ивентов для пользователей:

In [ ]:
table_sum = df_events.pivot_table(index=['uid','event_name'],  values='churn', aggfunc='count').reset_index()
table_sum = table_sum.rename(columns={'churn': 'count_event'})
table_sum.head()

,uid,event_name,count_event
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,spent_replay,4
1,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_level,20
2,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_replay,21
3,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_turns_purchase,3
4,00431e7e-d833-47c9-aae9-7ce21652902f,spent_replay,4


Выделим события для пользователей в течение первых 24 часов, и для 1-3 дней со старта.

In [ ]:
table_sum

,uid,event_name,count_event
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,spent_replay,4
1,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_level,20
2,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_replay,21
3,0018f5d2-d883-4ad2-9b96-29028069ee81,spent_turns_purchase,3
4,00431e7e-d833-47c9-aae9-7ce21652902f,spent_replay,4
...,...,...,...
5264,ff96e64e-8ebf-48e3-8d52-78f247468068,spent_replay,31
5265,ff96e64e-8ebf-48e3-8d52-78f247468068,spent_turns_purchase,3
5266,ffc552f6-b662-4b62-bf3b-188cf5c23760,spent_replay,4
5267,ffdb1aaf-6c57-4fb7-a3a9-da40857f9cf8,spent_replay,5


In [ ]:
table_events_24 = pd.merge(table_24, table_sum, how='inner', on='uid')
table_events_3 = pd.merge(table_3, table_sum, how='inner', on='uid')

Посмотрим на процент игр с использованнием бустеров, для этого добавим столбец отношения суммы бустеров к количеству игр, но не к максимальному уровню.

In [ ]:
table_events_24['percent'] = table_events_24['count_event'] / table_events_24['battle_start']
table_events_3['percent'] = table_events_3['count_event_y'] / table_events_3['battle_start']

Для каждого из типов ивентов построим гистограмму распределения этого отношения в зависимости от churn.

In [ ]:
pd.pivot_table(table_events_24, index='event_name',columns='churn', values='uid', aggfunc='nunique').reset_index()

churn,event_name,0,1
0,spent_level,284,876
1,spent_life_purchase,5,53
2,spent_replay,1254,1431
3,spent_turns_purchase,368,910


# **5.1 Связь прохождения уровней с расходом жизней на уровне - spent_replay**

In [ ]:
table_24_kogort = kogort_15(table_events_24)
table_3_kogort = kogort_15(table_events_3)

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_replay') &
                             (table_24_kogort['percent'] <= 1) &
                              (table_24_kogort['churn'] == 1)], 
              x="percent", color='level_group',
            barmode='group',nbins=5,
             title='With spent_replay for churn = 1')

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_replay') &
                             (table_24_kogort['percent'] <= 1) &
                              (table_24_kogort['churn'] == 0)], 
              x="percent", color='level_group',
            barmode='group',nbins=5,
             title='With spent_replay for churn = 0')

In [ ]:
px.histogram(table_3_kogort[(table_3_kogort['event_name'] == 'spent_replay')], 
              x="percent", color='level_group',
             title='Histogram for count for spend_replay',
             barmode='group', nbins=5)

**Выводы:** 

* Люди, которые забрасывают играть с churn=1 в среднем используют бустеры с процентом от 0.025 до 0.175.
* Люди которые дальше играют, имеют явный пик на 1. То есть используют этот тип бустера, в каждой игре.
* Для пользователей играющих больше чем сутки, процент спадает до 0.025-0.175.


# **5.2 Связь прохождения уровней с расходом хинта на уровне - spent_level**

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_level') &
                              (table_24_kogort['churn'] == 1)], 
              x="percent", color='level_group',
            barmode='group',nbins=5,
             title='Histogram for count for spend_level with churn = 1')

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_level') &
                              (table_24_kogort['churn'] == 0)], 
              x="percent", color='level_group',
            barmode='group',nbins=10,
             title='Histogram for count for spend_level with churn = 0')

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_level') &
                             (table_24_kogort['churn'] == 0)], 
              x="percent", color='level_group',
             title='Histogram for count for spend_level',
             barmode='group',nbins=5)

In [ ]:
px.histogram(table_events_3[table_events_3['event_name'] == 'spent_level'], 
              x="percent", color='churn',
             title='Histogram for count for spend_level',
             barmode='group')

**Выводы:**

* Упоминание данного ивента чаще регистрируется для людей с churn=1.  
* Данный тип бустера используется в среднем 0-0.04 процентов игр для людей с churn=1 и 0.02 - 0.2 людьми продолжающими играть.
* Также в данных встретился случай, когда пользователи, использовав этот бустер, были записаны повторно. Возможно опечатка, возможно этот бустер можно использовать несколько раз за игру.
* Для пользователей играющих больше суток распределение процентов такое же.


# **5.3 Связь прохождения уровней с покупкой ходов в гринде - spent_turns_purchase**

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_turns_purchase') &
                             (table_24_kogort['churn'] == 0)], 
              x="percent", color='level_group',
             title='Histogram for count for spent_turns_purchase with churn = 0',
             barmode='group',nbins=5)

In [ ]:
px.histogram(table_24_kogort[(table_24_kogort['event_name'] == 'spent_turns_purchase') &
                             (table_24_kogort['churn'] == 1)], 
              x="percent", color='level_group',
             title='Histogram for count for spent_turns_purchase with churn = 1',
             barmode='group',nbins=5)

In [ ]:
px.histogram(table_events_3[table_events_3['event_name'] == 'spent_turns_purchase'], 
              x="percent", color='churn',
             title='Histogram for count for spent_turns_purchase',
             barmode='group')

**Выводы:**

* Упоминание данного ивента чаще регистрируется для игроков, которые забрасывают играть, чем теми, кто остается .
* Процент использования данного бустера для людей с churn=1 составляет 0.005-0.045. Это верно для людей играющих сутки или больше.
* Процент использования данного бустера для людей с churn=0 составляет 0.045-0.135. Но данных для этой группы пользователей меньше.

# **5.4 Связь прохождения уровней с покупкой жизней - spent_life_purchase**

In [ ]:
px.histogram(table_events_24[table_events_24['event_name'] == 'spent_life_purchase'], 
              x="percent", color='churn',
             title='Histogram for count for spent_life_purchase',
             barmode='group')

In [ ]:
px.histogram(table_events_3[table_events_3['event_name'] == 'spent_life_purchase'], 
              x="percent", color='churn',
             title='Histogram for count for spent_life_purchase')

**Выводы:**
* Данный бустер используется реже всего. И чаще его используют люди, которые забрасывают играть.
* Выборка данных для данного бустера сликшмо мала, чтоб рассуждать о проценте использования.

# **6.1 Зависимость времени прохождения игры от когорты пользователей по уровням.**

In [ ]:
df_levels_new['delta_time'] == df_levels_new['delta_time'].dt.total_seconds()
df_levels_new['delta_time'] =df_levels_new['delta_time'].dt.seconds.astype('float64') /60

In [ ]:
df_temp = pd.merge(df_levels_new,table_24_kogort[['uid','level_group']],how='inner', on='uid')

In [ ]:
df_temp['level_group'] = df_temp['level_group'].astype('float64') 
df_temp_24 = df_temp[df_temp['24_hour'] == 1]

In [ ]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818579 entries, 0 to 818578
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   index                 818579 non-null  object         
 1   uid                   818579 non-null  object         
 2   created_at            818579 non-null  datetime64[ns] 
 3   event_name            818579 non-null  object         
 4   level_move            818579 non-null  object         
 5   Number_of_used_moves  400306 non-null  float64        
 6   target                400306 non-null  object         
 7   remaining_targets     393982 non-null  float64        
 8   reg_dt                818579 non-null  datetime64[ns] 
 9   churn                 818579 non-null  object         
 10  level                 782803 non-null  float64        
 11  delta_time            418273 non-null  float64        
 12  timedelta             818579 non-null  timed

In [ ]:
table_2 = df_temp_24[df_temp_24['event_name'] == 'battle_start'
                        ].pivot_table(index=['uid','churn', 'level_group'], 
                                      aggfunc={'level': 'max','delta_time': 'median'}).reset_index()
table_2.head(4)

,uid,churn,level_group,delta_time,level
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,0.0,0.700000,5.0
1,0018f5d2-d883-4ad2-9b96-29028069ee81,1,30.0,1.800000,44.0
2,00431e7e-d833-47c9-aae9-7ce21652902f,0,0.0,0.808333,4.0
3,0060a2dd-18d8-4122-96ca-92ae699a1a52,1,45.0,1.641667,58.0


In [ ]:
px.box(table_2, y='delta_time')

In [ ]:
px.histogram(table_2[table_2['delta_time'] < 3], x='delta_time', color='level_group', pattern_shape="churn",
             title='Histogram for delta_time',
             barmode='group',nbins=5)

In [ ]:
px.histogram(table_2[table_2['delta_time']>= 3], x='delta_time', color='level_group', pattern_shape="churn",
             title='Histogram for delta_time',
             barmode='group',nbins=5)

In [ ]:
px.scatter(table_2[table_2['delta_time'] < 3], x="level", y="delta_time", color='churn',
	         size="level_group", size_max=60)

In [ ]:
df_temp_24.head(1)

,index,uid,created_at,event_name,level_move,Number_of_used_moves,target,remaining_targets,reg_dt,churn,level,delta_time,timedelta,24_hour,level_group
0,24,ad1be1b6-e696-42d6-ac03-99be5803a14b,2021-03-13 03:38:18,battle_start,20,NaN,NaN,NaN,2021-03-13 03:36:08,0,1.0,1.866667,0 days 00:02:10,1.0,0.0


In [ ]:
table_3 = df_temp_24[df_temp_24['event_name'] == 'battle_start'
                        ].pivot_table(index=['uid','churn'], 
                                      aggfunc={'level': 'max','delta_time': 'sum'}).reset_index()
table_3.head(4)

,uid,churn,delta_time,level
0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,4.216667,5.0
1,0018f5d2-d883-4ad2-9b96-29028069ee81,1,360.750000,44.0
2,00431e7e-d833-47c9-aae9-7ce21652902f,0,4.233333,4.0
3,0060a2dd-18d8-4122-96ca-92ae699a1a52,1,473.850000,58.0


In [ ]:
px.box(table_3, y='delta_time', x='churn')

In [ ]:
px.bar(table_3, x='level', y='delta_time',
             title='Histogram for delta_time',
             barmode='group')

In [ ]:
px.scatter(table_3, x="level", y="delta_time", color='churn', size_max=60)

# **6.2 Зависимость количества попыток для игроков которые остались, и которые нет по когортам пользователей**

In [ ]:
table_raw = df_temp_24.pivot_table(index=['level','uid','churn', 'level_group'],columns='event_name',
                                      values='delta_time', aggfunc='count'
                                      ).reset_index()
table_raw.head(1)

event_name,level,uid,churn,level_group,battle_backtrack,battle_loss,battle_start,battle_win
0,1.0,000c51e7-db50-4f30-95d6-462be3a2a5d3,0,0.0,NaN,NaN,1.0,0.0


In [ ]:
table_short_start = table_raw.pivot_table(index=['level','churn'], values='battle_start', 
                                    aggfunc='describe').reset_index()
table_short_start_1 = table_short_start.drop(table_short_start[
                                                             table_short_start['75%'] == 1.0].index)
table_short_start_1

,level,churn,25%,50%,75%,count,max,mean,min,std
0,1.0,0,1.0,1.0,2.0,1254.0,6.0,1.573365,1.0,0.873824
1,1.0,1,1.0,3.0,3.0,1431.0,12.0,2.367575,1.0,1.065891
2,2.0,0,1.0,1.0,2.0,1104.0,6.0,1.608696,1.0,0.869007
3,2.0,1,1.0,3.0,3.0,1403.0,22.0,2.375624,1.0,1.146553
4,3.0,0,1.0,1.0,2.0,991.0,6.0,1.667003,1.0,0.862291
...,...,...,...,...,...,...,...,...,...,...
155,99.0,1,8.0,8.0,8.0,1.0,8.0,8.000000,8.0,NaN
156,100.0,1,4.0,4.0,4.0,1.0,4.0,4.000000,4.0,NaN
157,101.0,1,4.0,4.0,4.0,1.0,4.0,4.000000,4.0,NaN
158,102.0,1,4.0,4.0,4.0,1.0,4.0,4.000000,4.0,NaN


In [ ]:
px.bar(table_short_start[table_short_start['churn'] == 1], x='level', y='max')

In [ ]:
px.bar(table_short_start[table_short_start['churn'] == 0
                         ], x='level', y='max')

In [ ]:
table_short_loss = table_raw.pivot_table(index=['level','churn'], values='battle_loss', 
                                    aggfunc='describe').reset_index()
table_short_loss

,level,churn,25%,50%,75%,count,max,mean,min,std
0,0.0,0,0.0,0.0,0.0,219.0,0.0,0.0,0.0,0.0
1,0.0,1,0.0,0.0,0.0,834.0,0.0,0.0,0.0,0.0
2,1.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,1.0,1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2.0,0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
360,302.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
361,303.0,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
362,304.0,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
363,305.0,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [ ]:
px.box(table_short_start, y='max', x='churn', points="all")

In [ ]:
table_short_loss[table_short_loss['level'] < 150]

,level,churn,25%,50%,75%,count,max,mean,min,std
0,0.0,0,0.0,0.0,0.0,219.0,0.0,0.0,0.0,0.0
1,0.0,1,0.0,0.0,0.0,834.0,0.0,0.0,0.0,0.0
2,1.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,1.0,1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2.0,0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
203,145.0,1,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
204,146.0,1,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0
205,147.0,1,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0
206,148.0,1,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0


In [ ]:
px.box(table_short_loss[table_short_loss['level'] < 150], y='75%', x='churn')

Разница количества винов и стартов

In [ ]:
table_start = df_levels_new[df_levels_new['event_name'] == 'battle_start'
                        ].pivot_table(index='level', values='uid', 
                                      aggfunc='count').reset_index()
table_win = df_levels_new[df_levels_new['event_name'] == 'battle_win'
                        ].pivot_table(index='level', values='uid', 
                                      aggfunc='count').reset_index()  
table_merge = pd.merge(table_start, table_win, on='level')
table_merge['loss_count'] = table_merge['uid_x'] - table_merge['uid_y']                                     

In [ ]:
px.box(table_merge, y=['loss_count'], points="all")

In [ ]:
px.bar(table_merge, x='level', y='loss_count')

# **6.2 North Star Metric для 24 часов.**

In [ ]:
nsm_24 = pd.merge(df_24, df_levels[['uid', 'delta_time']], on='uid')


KeyError: ignored

In [ ]:
nsm_24.dropna(subset=['delta_time'])

,uid,created_at,event_name,level_move,Number_of_used_moves,target,remaining_targets,reg_dt,churn,timedelta,24_hour,delta_time
0,ad1be1b6-e696-42d6-ac03-99be5803a14b,2021-03-13 03:38:18,battle_start,20,NaN,NaN,NaN,2021-03-13 03:36:08,0,0 days 00:02:10,1,0 days 00:01:52
2,ad1be1b6-e696-42d6-ac03-99be5803a14b,2021-03-13 03:40:10,battle_win,20,8.0,73,112.0,2021-03-13 03:36:08,0,0 days 00:04:02,1,0 days 00:01:52
4,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:43:15,battle_start,20,NaN,NaN,NaN,2021-03-13 08:41:28,1,0 days 00:01:47,1,0 days 00:02:22
6,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:43:15,battle_start,20,NaN,NaN,NaN,2021-03-13 08:41:28,1,0 days 00:01:47,1,0 days 00:00:54
8,77ad3869-852d-4520-a1c4-5fc7c3fe7c98,2021-03-13 08:43:15,battle_start,20,NaN,NaN,NaN,2021-03-13 08:41:28,1,0 days 00:01:47,1,0 days 00:00:41
...,...,...,...,...,...,...,...,...,...,...,...,...
28568179,6869413d-8c8a-4364-ae0b-e9a7049ff1d5,2021-03-18 01:24:36,battle_start,27,NaN,NaN,NaN,2021-03-17 23:55:01,0,0 days 01:29:35,1,0 days 00:00:54
28568181,6869413d-8c8a-4364-ae0b-e9a7049ff1d5,2021-03-18 01:24:36,battle_start,27,NaN,NaN,NaN,2021-03-17 23:55:01,0,0 days 01:29:35,1,0 days 00:01:37
28568183,6869413d-8c8a-4364-ae0b-e9a7049ff1d5,2021-03-18 01:24:36,battle_start,27,NaN,NaN,NaN,2021-03-17 23:55:01,0,0 days 01:29:35,1,0 days 00:01:45
28568185,6869413d-8c8a-4364-ae0b-e9a7049ff1d5,2021-03-18 01:24:36,battle_start,27,NaN,NaN,NaN,2021-03-17 23:55:01,0,0 days 01:29:35,1,0 days 00:01:47


# **7. Планы и идеи, что еще можно посмотреть.**

Можно посмотреть на связь:

* проигрышей на стартовых уровнях и связь с отвалом игроков;
* крашей или проблем с записью данных и отвалом игроков;
* связь использованных бустеров на различных уровнях;
* когорт игроков и использованных бустеров;
* когорт игроков по использованным бустерам и провалам в играх.

Помимо этого есть часть игроков, которые так и не закончили одного уровня. Посмотреть на причины данного события.

